For previous derivation, pleae see handwritten notes. We come to an expression for the Hamiltonian in terms of operators that we can then solve computationally. Approach 1 involved expanding out the many gates in the Hamiltonian using Mathematica, but this approach overcomplicated things.

Approach 2: Solving system using OpenFermion

In [ ]:
# Install and import OpenFermion and Numpy
%%capture
try:
    import openfermion
except ImportError:
    !pip install git+https://github.com/quantumlib/OpenFermion.git@master#egg=openfermion
!pip install --upgrade numpy

In [ ]:
from openfermion.ops import FermionOperator
from openfermion.transforms import jordan_wigner
from openfermion.linalg import eigenspectrum

# Define the one-body terms
onebody1 = FermionOperator('3^ 3')
onebody2 = FermionOperator('4^ 4')
onebody3 = FermionOperator('5^ 5')
onebody4 = FermionOperator('0^ 0')
onebody5 = FermionOperator('1^ 1')
onebody6 = FermionOperator('2^ 3')

# Create full one body expression
onebody = onebody1 + onebody2 + onebody3 - onebody4 - onebody5 - onebody6

# Define two-body terms
twobody1 = FermionOperator('3^ 4^ 1 0')
twobody2 = FermionOperator('0^ 1^ 4 3')
twobody3 = FermionOperator('3^ 5^ 2 0')
twobody4 = FermionOperator('0^ 2^ 5 3')
twobody5 = FermionOperator('4^ 5^ 2 1')
twobody6 = FermionOperator('1^ 2^ 5 4')

# Create full two-body expression
twobody = twobody1 + twobody2 + twobody3 + twobody4 + twobody5 + twobody6

# Construct full Hamiltonian
Hfull = 0.5*onebody + 0.8*twobody
# Perform built-in JW transformation
Hjw = jordan_wigner(Hfull)
# Print eigenvalues of JW Hamiltonian
Hspectrum = eigenspectrum(Hjw)
print(Hspectrum)

[-1.78875056e+00-3.48495754e-77j -1.28062485e+00+0.00000000e+00j
 -1.01260204e+00+1.02695539e-83j -8.46585610e-01-7.98657881e-73j
 -8.46585610e-01+2.33711778e-68j -8.46585610e-01+1.77551658e-33j
 -8.46585610e-01-3.79676612e-83j -7.80624847e-01+2.63163068e-76j
 -5.00000000e-01-1.71495819e-76j -5.00000000e-01+0.00000000e+00j
 -5.00000000e-01+0.00000000e+00j -5.00000000e-01+3.10434554e-75j
 -2.76148516e-01-2.33994076e-84j -1.16344786e-16+1.20184833e-74j
 -5.57173082e-17-9.90054266e-72j -5.46821352e-17-9.63669219e-72j
 -6.71022710e-19-6.28598445e-73j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  2.83532226e-18+2.33112888e-17j  2.83532226e-18-2.33112888e-17j
  6.98228787e-18+0.000000

In [1]:
# Check solutions by eye against numerical solutions from original paper
eig1 = - (0.5 - ((1 + 3*(0.8**2)))**0.5)
eig2 = + (0.5 + ((1 + 3*(0.8**2)))**0.5)
eig3 = - (0.5 + ((1 + 3*(0.8**2)))**0.5)
eig4 = + (0.5 - ((1 + 3*(0.8**2)))**0.5)

print('Numerical Eigenvalues from Original Paper:')
print(eig1)
print(eig2)
print(eig3)
print(eig4)


Numerical Eigenvalues from Original Paper:
1.2088007490635064
2.2088007490635064
-2.2088007490635064
-1.2088007490635064


Solutions don't allign with numerical solutions, and not enough time to dive into full documentation to find out why, need new approach.

Approach 3: Tensor expansion of operators in Python

In [ ]:
import numpy as np

# Define X, Y, Z and I gates
i = 1j
X = np.array([[0, 1],[1, 0]])
Y = np.array([[0, -i],[i, 0]])
Z = np.array([[1, 0],[0, -1]])
I = np.array([[1, 0],[0, 1]])

# Define all needed versions of X gates, equating to a unique X
# gate for each qubit, using tensor product
X1 = np.kron(X, np.kron(I, np.kron(I, np.kron(I, np.kron(I, I)))))
X2 = np.kron(I, np.kron(X, np.kron(I, np.kron(I, np.kron(I, I)))))
X3 = np.kron(I, np.kron(I, np.kron(X, np.kron(I, np.kron(I, I)))))
X4 = np.kron(I, np.kron(I, np.kron(I, np.kron(X, np.kron(I, I)))))
X5 = np.kron(I, np.kron(I, np.kron(I, np.kron(I, np.kron(X, I)))))
X6 = np.kron(I, np.kron(I, np.kron(I, np.kron(I, np.kron(I, X)))))

# Define all needed Y gates
Y1 = np.kron(Y, np.kron(I, np.kron(I, np.kron(I, np.kron(I, I)))))
Y2 = np.kron(I, np.kron(Y, np.kron(I, np.kron(I, np.kron(I, I)))))
Y3 = np.kron(I, np.kron(I, np.kron(Y, np.kron(I, np.kron(I, I)))))
Y4 = np.kron(I, np.kron(I, np.kron(I, np.kron(Y, np.kron(I, I)))))
Y5 = np.kron(I, np.kron(I, np.kron(I, np.kron(I, np.kron(Y, I)))))
Y6 = np.kron(I, np.kron(I, np.kron(I, np.kron(I, np.kron(I, Y)))))

# Define all Z gates
Z1 = np.kron(Z, np.kron(I, np.kron(I, np.kron(I, np.kron(I, I)))))
Z2 = np.kron(I, np.kron(Z, np.kron(I, np.kron(I, np.kron(I, I)))))
Z3 = np.kron(I, np.kron(I, np.kron(Z, np.kron(I, np.kron(I, I)))))
Z4 = np.kron(I, np.kron(I, np.kron(I, np.kron(Z, np.kron(I, I)))))
Z5 = np.kron(I, np.kron(I, np.kron(I, np.kron(I, np.kron(Z, I)))))
Z6 = np.kron(I, np.kron(I, np.kron(I, np.kron(I, np.kron(I, Z)))))

# Generate all creation operators using expression from notes
create1 = 0.5 * (X1 - i*Y1)
create2 = 0.5 * (X2 - i*Y2)@Z1
create3 = 0.5 * (X3 - i*Y3)@Z2@Z1
create4 = 0.5 * (X4 - i*Y4)@Z3@Z2@Z1
create5 = 0.5 * (X5 - i*Y5)@Z4@Z3@Z2@Z1
create6 = 0.5 * (X6 - i*Y6)@Z5@Z4@Z3@Z2@Z1

# Generate all annihilation operators
destroy1 = 0.5 * (X1 + i*Y1)
destroy2 = 0.5 * (X2 + i*Y2)@Z1
destroy3 = 0.5 * (X3 + i*Y3)@Z2@Z1
destroy4 = 0.5 * (X4 + i*Y4)@Z3@Z2@Z1
destroy5 = 0.5 * (X5 + i*Y5)@Z4@Z3@Z2@Z1
destroy6 = 0.5 * (X6 + i*Y6)@Z5@Z4@Z3@Z2@Z1

# Define one-body term, derived from notes
Honebody = Z1 + Z2 + Z3 - Z4 - Z5 - Z6

# Define two-body term, derived from notes
Htwobody = ((create4@create5@destroy2@destroy1) + (create1@create2@destroy5@destroy4)
            + (create4@create6@destroy3@destroy1) + (create1@create3@destroy6@destroy4)
            + (create5@create6@destroy3@destroy2) + (create2@create3@destroy6@destroy5))

# Construct full Hamiltonian and print eigenvalues
Hfull = 0.25 * Honebody + 0.8 * Htwobody
np.set_printoptions(linewidth=np.inf)
eigs = np.linalg.eigvals(Hfull)

eig1 = - (0.5 - ((1 + 3*(0.8**2)))**0.5)
eig2 = + (0.5 + ((1 + 3*(0.8**2)))**0.5)
eig3 = - (0.5 + ((1 + 3*(0.8**2)))**0.5)
eig4 = + (0.5 - ((1 + 3*(0.8**2)))**0.5)


# Check list of calculated eigenvalues and print True when a given eigenvalue
# is equal, to a certain tolerance, to the numerical eigenvalue from original
# paper
numericaleigs = [eig1, eig2, eig3, eig4]

print('Numerical Eigenvalues from Original Paper:')
for eig in numericaleigs:
    print(eig)

print('\nCalculated Eigenvalues:')
for eig in eigs:
    print(eig)

print('\nCheck if found solutions match with numerical solutions:')
tolerance = 1e-6
matches = [any(np.isclose(eig, calc, atol=tolerance) for calc in eigs) for eig in numericaleigs]
print(matches)

Numerical Eigenvalues from Original Paper:
1.2088007490635064
2.2088007490635064
-2.2088007490635064
-1.2088007490635064

Calculated Eigenvalues:
(-1.2806248474865698+0j)
(1.2806248474865698+0j)
(-2.2088007490635078+0j)
(0.49999999999999994+0j)
(1.208800749063506+0j)
(0.5+0j)
(1.2806248474865698+0j)
(-1.2806248474865698+0j)
(-1.2806248474865698+0j)
(1.2806248474865698+0j)
(-1.2806248474865698+0j)
(1.2806248474865698+0j)
(2.2088007490635078+0j)
(-0.4999999999999997+0j)
(-1.208800749063506+0j)
(1.2806248474865696+0j)
(-1.2806248474865698+0j)
(1.2806248474865694+0j)
(-1.2806248474865698+0j)
(-0.49999999999999994+0j)
0j
(0.5+0j)
(0.5+0j)
(0.5+0j)
(-0.5+0j)
0j
0j
(0.5+0j)
0j
(0.5+0j)
(-0.5+0j)
0j
0j
(0.5+0j)
0j
(0.5+0j)
(-0.5+0j)
(-0.5+0j)
0j
0j
0j
(0.5+0j)
(-0.5+0j)
0j
0j
0j
(0.5+0j)
(0.5+0j)
(-0.5+0j)
0j
(-0.5+0j)
0j
0j
(0.5+0j)
(-0.5+0j)
0j
(-0.5+0j)
0j
0j
(0.5+0j)
(-0.5+0j)
(-0.5+0j)
(-0.5+0j)
0j

Check if found solutions match with numerical solutions:
[True, True, True, True]


Calculated solutions allign with numerical solutions, so JW transformed Hamiltonian is accurate. We can also see the eigenvalues from the N = 2 model, [-1.28062485+0.j,  1.28062485+0.j].

